In [1]:
# !aws codeartifact login --tool pip --domain cirrus-ml-ds-domain --domain-owner 813736554012 --repository cirrus-ml-ds-shared-repo
# !pip install awswrangler
# !pip install lightgbm
# !pip install category_encoders
# !pip install imbalanced-learn


In [2]:
import boto3
import awswrangler as wr
import pandas as pd
from lightgbm import LGBMClassifier
import numpy as np

boto3.setup_default_session(region_name="eu-north-1")

In [3]:
df = wr.athena.read_sql_query('SELECT * FROM "customerone_mock_data_rl"."master";', 
                              database="customerone_mock_data_rl",
                              workgroup="dev-ds-athena-workgroup")
df

,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0
0,100116,2021-07-14,<NA>,49,26,U,North Julie,Uruguay,3,Bad,...,NaN,NaN,1.000000,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
1,100141,2021-05-28,<NA>,21,54,U,Bradleyshire,Swaziland,1,Bad,...,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,xsell_cust_voice_to_fixed
2,100154,2021-05-07,<NA>,55,73,U,Kyleview,Comoros,3,Bad,...,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,xsell_cust_voice_to_fixed
3,100178,2021-09-16,<NA>,40,74,F,Montoyaberg,Jamaica,3,Good,...,1.000000,NaN,1.000000,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
4,100185,2021-09-05,<NA>,48,69,M,North Elizabeth,Cyprus,4,Bad,...,NaN,NaN,1.000000,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40204,10090,2021-05-28,<NA>,47,0,U,North Tammyburgh,Ecuador,1,Bad,...,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40205,100909,2021-09-17,<NA>,44,16,F,Duffyport,France,2,Bad,...,1.000000,1.0,NaN,NaN,0,0,0.0,0.0,0.0,xsell_cust_voice_to_fixed
40206,100910,2021-05-31,<NA>,36,88,F,Port Daniel,Azerbaijan,3,Good,...,NaN,NaN,0.309677,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40207,10097,2021-06-19,<NA>,60,19,M,West Connieport,Australia,2,Bad,...,NaN,NaN,1.000000,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed


In [4]:
# 74 columns
# [print(f"\n** {col}:\n{df[col].value_counts(dropna=False)} \n") for col in df.columns];

# 0- configurations

In [5]:
target_col_name = 'tgt_xsell_cust_voice_to_fixed'


# 1- sample_training_df()

In [6]:
df_training = df[df['tgt_xsell_cust_voice_to_fixed'].notnull()]

In [7]:
df_training

,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0
9,100254,2021-05-26,0,42,68,U,East Jacquelineshire,Liechtenstein,0,Bad,...,1.0,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
43,100249,2021-06-25,0,24,91,U,South Marissa,Qatar,3,Bad,...,NaN,NaN,1.0,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed
44,100249,2021-06-25,0,24,91,U,South Marissa,Qatar,3,Bad,...,NaN,NaN,1.0,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed
45,100249,2021-06-25,0,24,91,U,South Marissa,Qatar,3,Bad,...,NaN,NaN,1.0,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed
51,100307,2021-08-01,0,52,41,M,Holderburgh,Seychelles,4,Bad,...,NaN,3.444444,1.0,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40183,10060,2021-06-11,0,40,64,F,Cruzview,Saint Martin,1,Bad,...,NaN,NaN,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed
40185,100642,2021-06-21,0,50,46,M,Rollinsville,Germany,2,Bad,...,1.0,NaN,NaN,NaN,0,0,272.358113,0.0,0.0,xsell_cust_voice_to_fixed
40201,100899,2021-07-14,0,64,44,U,Melissashire,British Indian Ocean Territory (Chagos Archipe...,0,Good,...,1.0,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40202,100899,2021-07-14,0,64,44,U,Melissashire,British Indian Ocean Territory (Chagos Archipe...,0,Good,...,1.0,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed


In [8]:
# drop_invalid_features
df_training_idx= df_training.reset_index(drop=True)
print(f"{df_training.shape}, {df_training_idx.shape}")

(5896, 74), (5896, 74)


# 6 & 8- splitting

In [9]:
# {{model}}.training.splitting_params:
#   splitter: customerone.pipelines.telco.test_train_splitter.OutOfTimeSplitter
#   splitter_args: # Arguments to be passed into the splitter, see the splitter docs for details
#     n_splits: 1 # The n_splits argument controls the number of iterations
#     test_size: 0.2
#     date_col: "current_dt"
#     verbose: 1
#     random_state: 42
#   group_col_name: "customer_id" # column be useded as a group vector, if the selected spliter uses the group parameter. See the splitter docs for details
#   iteration_col_name: iteration_id # Name of the iteration ID column to be added
#   split_col_name: split # Name of the TRAIN/TEST/etc. split column

# {{model}}.training.subsplitting_params:
#   splitter: sklearn.model_selection.StratifiedShuffleSplit
#   splitter_args: # Arguments to be passed into the splitter, see the splitter docs for details
#     test_size: 0.1
#     random_state: 42 # n_splits is fixed to be 1 for sub-splitters. Any higher value will be ignored
#   source_split: TRAIN # The set to split
#   target_splits: # The two sets to split the source set into. Must be two targets
#     - TRAIN
#     - CAL

In [10]:
from utils_split import make_splits, split_data, make_subsplit

In [11]:
split_train_test = make_splits(df_training_idx, target_col_name)



_extract_group_column_arg: group_col_name=customer_id
_extract_group_column_arg: target_col_name=tgt_xsell_cust_voice_to_fixed

Split 0
	train: 	2021-05-25 -> 2021-08-14	shape: (4654, 76)	ratio: 0.79
	test: 	2021-08-15 -> 2021-09-09	shape: (1242, 76)	ratio: 0.21


In [12]:
print(f"{df_training.shape}, {df_training_idx.shape}, {split_train_test.shape}")
# (5896, 74), (5896, 76), (5896, 76)

(5896, 74), (5896, 76), (5896, 76)


In [13]:
split_train_test['split'].value_counts(dropna=False)
# TRAIN    4654
# TEST     1242
# Name: split, dtype: int64

TRAIN    4654
TEST     1242
Name: split, dtype: int64

In [14]:
df_training_idx['split'].value_counts(dropna=False)
# NaN    5896
# Name: split, dtype: int64

NaN    5896
Name: split, dtype: int64

In [15]:
split_train_test['iteration_id'].value_counts(dropna=False)


0    5896
Name: iteration_id, dtype: int64

In [16]:
splitter_args = {'test_size': 0.1, 'random_state': 42}
subsplit_params = {"iteration_col_name": 'iteration_id', "split_col_name": "split", 'source_split': 'TRAIN', 'target_splits':['TRAIN', 'CAL']}

split_train_test_cal = make_subsplit(split_train_test, subsplit_params, splitter_args, target_col_name)
split_train_test_cal



_extract_group_column_arg: group_col_name=None
_extract_group_column_arg: target_col_name=tgt_xsell_cust_voice_to_fixed


,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0,iteration_id,split
0,100254,2021-05-26,0,42,68,U,East Jacquelineshire,Liechtenstein,0,Bad,...,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
1,100249,2021-06-25,0,24,91,U,South Marissa,Qatar,3,Bad,...,1.0,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed,0,CAL
2,100249,2021-06-25,0,24,91,U,South Marissa,Qatar,3,Bad,...,1.0,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
3,100249,2021-06-25,0,24,91,U,South Marissa,Qatar,3,Bad,...,1.0,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
4,100307,2021-08-01,0,52,41,M,Holderburgh,Seychelles,4,Bad,...,1.0,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891,10060,2021-06-11,0,40,64,F,Cruzview,Saint Martin,1,Bad,...,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
5892,100642,2021-06-21,0,50,46,M,Rollinsville,Germany,2,Bad,...,NaN,NaN,0,0,272.358113,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
5893,100899,2021-07-14,0,64,44,U,Melissashire,British Indian Ocean Territory (Chagos Archipe...,0,Good,...,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
5894,100899,2021-07-14,0,64,44,U,Melissashire,British Indian Ocean Territory (Chagos Archipe...,0,Good,...,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN


In [17]:
# split_train_test['split'].value_counts(dropna=False)
# TRAIN    4654
# TEST     1242
split_train_test_cal['split'].value_counts(dropna=False)

TRAIN    4188
TEST     1242
CAL       466
Name: split, dtype: int64

# 7- feature types

In [18]:
from utils_determine_feature_type import determine_feature_data_types

spine_params_determine_feature_data_types = {'keys': ['customer_id'], 
                'date_column': 'current_dt', 
                'product_holdings_filter': {'product_category': 'fixedbroadband'}, 
                'is_deepsell': 'N'}

target_params_determine_feature_data_types = {'target_variable_column': 'tgt_xsell_cust_voice_to_fixed'
                 , 'lead_time_window': '1d'
                 , 'target_window': '45d'
                 , 'product_activation_filter': {'product_sub_category': 'voice'}
                 , 'campaign_keys': ['customer_id']
                 , 'campaign_filter': {'campaign_name':["C-452-O-06 Korsförsäljning Telia Life 2.0 - Sälja mobilt"
                                            , "b2c_cross-sell_pp_crossSellPpToBb"
                                            , "b2c_cross-sell_pp_crossSellPpToBb_oldContent"
                                            , "b2c_cross-sell_pp_crossSellPpToBb_REM1"
                                            , "C-700-O-03 Cross-sell Mobile to Broadband customers (TM only)"
                                            , "C-700-O-01 Cross-sell Mobile to Broadband customers (A)"
                                            , "C-752-O GEOF 2021 - X-sell PP"
                                            , "C-652-O Black Friday Erbjudande 2019  Mobilt till BB-kund - activity 1"
                                            , "C-700-O-02 Cross-sell Mobile to Broadband customers (B)"
                                            , "b2c_cross-sell_Pp_PpToBb_default"
                                            , "C-752-O GEOF 2021 - Xsell PP"
                                            , "b2c_cross-sell_Pp_PpToBb_simOnly"
                                            , "b2c_cross-sell_Pp_PpToBb_samS215G"
                                            , "b2c_cross-sell_Pp_PpToBb_iphone12Mini"
                                            , "b2c_cross-sell_Pp_PpToBb_iphoneSE"
                                            , "b2c_cross-sell_Pp_PpToBb_iphone12"
                                            , "b2c_cross-sell_Pp_PpToBb_samS20FE5G"
                                            , "b2c_cross-sell_Pp_PpToBb_sonyXp10lll"
                                            , "b2c_cross-sell_Pp_PpToBb_default_short_8pm"
                                            , "b2c_cross-sell_Pp_PpToBb_default_8pm"]
                                       , 'customer_actioned_flg_column': {'Email':'actioned_ind'}}}
                 

feature_dict = determine_feature_data_types(df_training, spine_params_determine_feature_data_types, target_params_determine_feature_data_types)
print(f"len(feature_dict)={len(feature_dict)}, \nlen(feature_dict['numeric'])={len(feature_dict['numeric'])}, \nlen(feature_dict['categorical'])={len(feature_dict['categorical'])}")
feature_dict
# actual data: len(categorical)=7 , len(numerical)=150

len(feature_dict)=3, 
len(feature_dict['numeric'])=62, 
len(feature_dict['categorical'])=3


{'numeric': ['cust_prd_hld_d_fixedbroadband_active_30_to_60_days_avg_val',
  'cust_prd_hld_d_tvchannelpackage_closed_0_to_30_days_avg_val',
  'ci_d_variation_sum_total_num_of_interactions_last_30_to_60_days_cnt',
  'ci_d_total_num_of_interactions_weekend_last_0_to_30_cnt',
  'ci_d_sum_total_compl_last_0_to_30_days_cnt',
  'cust_prd_hld_d_mobilevoicesubscription_closed_val',
  'cust_prd_hld_d_vas_closed_0_to_30_days_avg_val',
  'cust_prd_hld_d_postpaid_tvchannelpackage_cnt',
  'cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val',
  'ci_d_sum_compl_fraga_last_0_to_60_days_cnt',
  'cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val',
  'cust_prd_hld_d_fixedbroadband_active_val',
  'cust_prd_hld_d_fixedbroadband_active_0_to_30_days_avg_val',
  'cust_prd_hld_d_vas_active_0_to_30_days_avg_val',
  'dmgrphc_b_no_of_dependents_val',
  'ci_d_sum_compl_fraga_last_0_to_30_days_cnt',
  'rev_m_eom_total_bill_amt_m2_val',
  'ci_d_sum_cmpl_over_cont

# 10- preprocessing_pipeline_step

In [19]:
#                 node(
#                     func=preprocessing_pipeline_step,
#                     inputs={
#                         "params": f"{param_str}.preprocessing",
#                         "categorical": f"{model_name}.in_memory.categorical_features",
#                         "numerical": f"{model_name}.in_memory.numerical_features",
#                     },
#                     outputs=f"{catalog_str}.preprocessing_step",
#                     name=f"define_preprocessing",
#                     tags=["training: model"],
#                 )
            

# categorical_features = {'categorical': {'impute': {'class': 'customerone_lib.common.pkgs.predictive_modeling.preprocessing.PandasSimpleImputer',
#                                                     'args': {'strategy': 'most_frequent'}},
#                                         'encoder': {'class': 'category_encoders.target_encoder.TargetEncoder'}}}

# numerical_features = {'numerical':{'imputer': {'class': 'customerone_lib.common.pkgs.predictive_modeling.preprocessing.PandasSimpleImputer',
#                                                'args': {'strategy': 'mean'}}}}

# params_preprocessing = {'categorical': {'impute': PandasSimpleImputer(strategy='most_frequent'),
#                                         'encoder': TargetEncoder()}, 
#                         'numerical': {'imputer': PandasSimpleImputer()}}

%load_ext autoreload 
%autoreload 2
from utiles_preprocessing import preprocessing_pipeline_step


preprocessing_step = preprocessing_pipeline_step(feature_dict['categorical'], feature_dict['numeric'])
preprocessing_step

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


 ****** PandasSimpleImputer > fit ******** 
 ****** preprocessing_pipeline_step  ******** 


 len preprocessing = 2


ColumnTransformer(transformers=[('categorical',
                                 Pipeline(steps=[('impute',
                                                  PandasSimpleImputer(strategy='most_frequent')),
                                                 ('encoder', TargetEncoder())]),
                                 ['rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg',
                                  'dmgrphc_b_zip_code_cd',
                                  'rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg']),
                                ('numerical',
                                 Pipeline(steps=[('imputer',
                                                  PandasSimpleImputer())]),
                                 ['rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg',
                                  'dmgrphc_b_zip_code_cd',
                                  'rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg'])])

# before vs after imputation?!

# 11 & 2 estimator

In [20]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
# from sklearn.pipeline import Pipeline

def model_estimator(**kwargs):
# def model_estimator(params):
# def model_estimator(params: Dict[str, Any],) -> BaseEstimator:
    """
    Define a estimator

    `params` expects the following key value pairs:
        `estimator`: path to estimator class
        `args`: (Optional) arguments to be passed when initiating the estimator object

    or 

    `params` expects the following key value pairs:
        `pipeline`: path to pipeline class
        `args`: arguments to be passed when initiating the pipeline object.
            Must at least contain the `steps` key
            `steps`: steps defining transformers (implementing fit/transform/fit_resample), 
                in the order in which they are chained, with the last object being an estimator.  
                `step_name_1`: name of the first step
                    `transformer`: Path to transformer/estimator class
                    `args`: arguments to be passed to transformer/estimator instantiation
                `step_name_2`: name of the second step, follows the same structure as step one
                    ...

    Args:
        params: dictonary with the keys {'estimator': string, 'args': dict} 
            or {'pipeline': string, 'args': {'steps': dict}}

    Returns:
        estimator
    """

    estimator_pipeline = Pipeline(steps=[('over_sampler', SMOTE(random_state=42)),
                                         ('estimator', LGBMClassifier())])
    if len(kwargs):
        estimator_pipeline.steps.insert(kwargs.get('index'), (kwargs.get('step_name'), kwargs.get('step')))
    return estimator_pipeline

In [21]:
# estimator_params = {'pipeline': 'imblearn.pipeline.Pipeline', 
#                     'args': {'steps': {'over_sampler': {'class': 'imblearn.over_sampling.SMOTE', 
#                                                         'args': {'random_state': 42}}, 
#                                        'estimator': {'class': 'lightgbm.LGBMClassifier'}}}}
add_step = {'step':preprocessing_step, 'index': 0, 'step_name':'preprocessing'}
estimator = model_estimator(**add_step)
estimator

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('impute',
                                                                   PandasSimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   TargetEncoder())]),
                                                  ['rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg',
                                                   'dmgrphc_b_zip_code_cd',
                                                   'rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   PandasSimpleImputer())]),
                

In [22]:
#   estimator = Pipeline(steps=[('preprocessing',
#                  ColumnTransformer(transformers=[('categorical',
#                                                   Pipeline(steps=[('impute',
#                                                                    PandasSimpleImputer(strategy='most_frequent')),
#                                                                   ('encoder',
#                                                                    TargetEncoder())]),
#                                                   ['dmgrphc_b_gender_typ',
#                                                    'dmgrphc_b_habits_desc_txt',
#                                                    'dmgrphc_b_age_bucket_txt',
#                                                    'dmgrphc_b_city_txt',
#                                                    'dmgrphc_b_lifestyle_desc_txt',
#                                                    'rev_m_bill_shock_eom_total_bill_amt...
#                                                    'product_holding_mobilebroadbandsubscription_active_30_to_60_days_avg',
#                                                    'product_holding_fixedbroadband_active_0_to_30_over_30_to_60_days_avg',
#                                                    'ci_d_sum_cmpl_over_contacts_in_0_30_days',
#                                                    'product_holding_fixedbroadband_closed_0_to_30_over_30_to_60_days_avg',
#                                                    'num_distinct_campaign_last_30_days', ...])])),
#                 ('over_sampler', SMOTE(random_state=42)),
#                 ('estimator', LGBMClassifier())])   


# node(
#     func=model_estimator,
#     inputs={
#         "params": f"{param_str}.estimator",
#     },
#     outputs=f"{model_name}.in_memory.estimator",
#     name=f"define_estimator",
#     tags=["training: model"],
# ),
# node(
#     func=partial(add_step, step_name="preprocessing"),
#     inputs={
#         "pipeline": f"{model_name}.in_memory.estimator",
#         "step": f"{catalog_str}.preprocessing_step"
#     },
#     outputs=f"{catalog_str}.estimator",
#     name=f"add_preprocessing_step_to_estimator",
#     tags=["training: model"],
# )
        
        
# {{model}}.training.estimator:
#   pipeline: imblearn.pipeline.Pipeline
#   args:
#     steps:
#       over_sampler: 
#         class: imblearn.over_sampling.SMOTE
#         args:
#           random_state: 42
#       estimator: 
#         class: lightgbm.LGBMClassifier        


# 12- tune_parameters

In [23]:
%load_ext autoreload 
%autoreload 2
from utils_tune_parameters import tune_parameters


tuning_params = {'tuner': 'sklearn.model_selection.RandomizedSearchCV', 
                 'tuner_args': {'scoring': 'roc_auc', 'verbose': 2, 'n_iter': 10, 
                                'param_distributions': {'preprocessing__categorical__encoder__min_samples_leaf': [100, 500, 1000, 5000, 10000],
                                                         'preprocessing__categorical__encoder__smoothing': [1.0, 5.0, 10.0, 25.0], 
                                                         'over_sampler__sampling_strategy': [0.5, 0.35, 0.25, 0.15, 0.1, 'not minority'], 
                                                         'estimator__boosting_type': ['gbdt', 'dart'], 
                                                         'estimator__num_leaves': [20, 31, 50, 100], 
                                                         'estimator__min_child_samples': [100, 250, 500, 1000], 
                                                         'estimator__min_child_weight': [0.01, 0.001], 
                                                         'estimator__max_depth': [5, 10, 15, -1], 
                                                         'estimator__learning_rate': [0.2, 0.15, 0.1, 0.05, 0.01], 
                                                         'estimator__n_estimators': [50, 100, 150], 
                                                         'estimator__colsample_bytree': [0.8, 1.0], 
                                                         'estimator__subsample': [0.8, 1.0], 
                                                         'estimator__subsample_freq': [1], 
                                                         'estimator__reg_alpha': [0.0, 0.1],
                                                         'estimator__reg_lambda': [0.0, 0.1],
                                                         'estimator__min_split_gain': [0.0, 0.1],
                                                         'estimator__class_weight': ["balanced", []],
                                                        }}, 
                 'cv': 'test_train_splitter.OutOfTimeSplitter', 
                 'cv_args': {'n_splits': 3, 'test_size': 0.45, 'date_col': 'current_dt', 'random_state': 42, 'verbose': 1}}

features = feature_dict['categorical'] + feature_dict['numeric']

splitting_params = {'splitter': 'test_train_splitter.OutOfTimeSplitter', 
                    'splitter_args': {'n_splits': 1, 'test_size': 0.2, 'date_col': 'current_dt', 'verbose': 1, 'random_state': 42}, 
                    'group_col_name': 'customer_id', 
                    'iteration_col_name': 'iteration_id', 
                    'split_col_name': 'split'}

target = 'tgt_xsell_cust_voice_to_fixed'

tuned_parameters, best_tuning_score = tune_parameters(split_train_test_cal, estimator, tuning_params, features, splitting_params, target_col_name)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

 OutOfTimeSplitter.split > split=0
	train: 	2021-05-25 -> 2021-07-08	shape: (2261, 76)	ratio: 0.54
	test: 	2021-07-09 -> 2021-07-19	shape: (675, 76)	ratio: 0.16

 OutOfTimeSplitter.split > split=1
	train: 	2021-05-25 -> 2021-07-18	shape: (2879, 76)	ratio: 0.69
	test: 	2021-07-19 -> 2021-08-02	shape: (724, 76)	ratio: 0.17

 OutOfTimeSplitter.split > split=2
	train: 	2021-05-25 -> 2021-08-01	shape: (3558, 76)	ratio: 0.85
	test: 	2021-08-02 -> 2021-08-14	shape: (630, 76)	ratio: 0.15
Fitting 3 folds for each of 10 candidates, totalling 30 fits
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
[CV] END estimator__boosting_type=dart, estimator__class_weight=[], estimator__colsample_bytree=1.0, estimator__learning_rate=0.05, estimator__max_depth=15, estimato

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
[CV] END estimator__boosting_type=dart, estimator__class_weight=[], estimator__colsample_bytree=1.0, estimator__learning_rate=0.05, estimator__max_depth=15, estimator__min_child_samples=1000, estimator__min_child_weight=0.01, estimator__min_split_gain=0.0, estimator__n_estimators=50, estimator__num_leaves=50, estimator__reg_alpha=0.0, estimator__reg_lambda=0.1, estimator__subsample=1.0, estimator__subsample_freq=1, over_sampler__sampling_strategy=0.5, preprocessing__categorical__encoder__min_samples_leaf=5000, preprocessing__categorical__encoder__smoothing=1.0; total time=   0.1s
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
[CV] END esti

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
[CV] END estimator__boosting_type=gbdt, estimator__class_weight=balanced, estimator__colsample_bytree=0.8, estimator__learning_rate=0.15, estimator__max_depth=15, estimator__min_child_samples=250, estimator__min_child_weight=0.01, estimator__min_split_gain=0.1, estimator__n_estimators=50, estimator__num_leaves=20, estimator__reg_alpha=0.1, estimator__reg_lambda=0.1, estimator__subsample=1.0, estimator__subsample_freq=1, over_sampler__sampling_strategy=0.25, preprocessing__categorical__encoder__min_samples_leaf=10000, preprocessing__categorical__encoder__smoothing=10.0; total time=   0.1s
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
[CV] END estimator__boosting_type=gbdt, estimator__class_weight=balanced, estimator__colsample_bytree=0.8

...

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


[CV] END estimator__boosting_type=dart, estimator__class_weight=balanced, estimator__colsample_bytree=1.0, estimator__learning_rate=0.1, estimator__max_depth=5, estimator__min_child_samples=500, estimator__min_child_weight=0.001, estimator__min_split_gain=0.0, estimator__n_estimators=100, estimator__num_leaves=100, estimator__reg_alpha=0.0, estimator__reg_lambda=0.1, estimator__subsample=1.0, estimator__subsample_freq=1, over_sampler__sampling_strategy=0.5, preprocessing__categorical__encoder__min_samples_leaf=1000, preprocessing__categorical__encoder__smoothing=25.0; total time=   0.1s
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
[CV] END estimator__boosting_type=dart, estimator__class_weight=balanced, estimator__colsample_bytree=1.0, estimator__learning_rate=0.1, estimator__max_depth=5, estimator__min_child_samples=500, estimator__

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
[CV] END estimator__boosting_type=dart, estimator__class_weight=balanced, estimator__colsample_bytree=1.0, estimator__learning_rate=0.1, estimator__max_depth=5, estimator__min_child_samples=500, estimator__min_child_weight=0.001, estimator__min_split_gain=0.0, estimator__n_estimators=100, estimator__num_leaves=100, estimator__reg_alpha=0.0, estimator__reg_lambda=0.1, estimator__subsample=1.0, estimator__subsample_freq=1, over_sampler__sampling_strategy=0.5, preprocessing__categorical__encoder__min_samples_leaf=1000, preprocessing__categorical__encoder__smoothing=25.0; total time=   0.1s
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
[CV] END estimator__boosting_type=dart, estimator__class_weight=balanced, estimator__colsample_bytree=1.0,

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
3 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/imblearn/pipeline.py", line 277, in fit
  

In [24]:
# node(
#     func=tune_parameters,
#     inputs={
#         "data": f"{catalog_str}.split_train_test_cal",
#         "estimator": f"{catalog_str}.estimator",
#         "tuning_params": f"{param_str}.tuning_params",
#         "features": f"{model_name}.in_memory.features",
#         "splitting_params": f"{param_str}.splitting_params",
#         "target_col_name": f"{param_str}.target",
#     },
#     outputs=[
#         f"{catalog_str}.tuned_parameters",
#         f"{catalog_str}.best_tuning_score"
#     ],
#     name=f"tune_model_params_for_training",
#     tags=["training: model"],
# )
    
    
# # No features are specified for this model as they are found automatically in feature selection
# {{model}}.training.tuning_params:
#   tuner: sklearn.model_selection.RandomizedSearchCV
#   tuner_args:
#     scoring: roc_auc
#     verbose: 2
#     n_iter: 10
#     # if estimator is a sklearn/imblearn pipeline, then the param_distributions 
#     # needs to be prefixed with step name, e.g '{step_name}__{parameter}'
#     param_distributions:
#       preprocessing__categorical__encoder__min_samples_leaf: [100, 500, 1000, 5000, 10000]
#       preprocessing__categorical__encoder__smoothing: [1.0, 5.0, 10.0, 25.0] # float is needed
#       # 'not minority' acts as a 'passthrough' for SMOTE
#       over_sampler__sampling_strategy: [0.5, 0.35, 0.25, 0.15, 0.1, "not minority"]
#       estimator__boosting_type: ["gbdt", "dart"]
#       estimator__num_leaves: [20, 31, 50, 100]
#       estimator__min_child_samples: [100, 250, 500, 1000]
#       estimator__min_child_weight: [0.01, 0.001]
#       estimator__max_depth: [5, 10, 15, -1]
#       estimator__learning_rate: [0.2, 0.15, 0.1, 0.05, 0.01]
#       estimator__n_estimators: [50, 100, 150]
#       estimator__colsample_bytree: [0.8, 1.0]
#       estimator__subsample: [0.8, 1.0]
#       estimator__subsample_freq: [1]
#       estimator__reg_alpha: [0.0, 0.1]
#       estimator__reg_lambda: [0.0, 0.1]
#       estimator__min_split_gain: [0.0, 0.1]
#       estimator__class_weight: ["balanced", []] # empty list [] acts as None, all classes gets weight one
#   cv: customerone.pipelines.telco.test_train_splitter.OutOfTimeSplitter
#   cv_args:’”‘
#     n_splits: 3
#     test_size: 0.45
#     date_col: current_dt
#     random_state: 42
#     verbose: 1    

        
        
   

In [25]:
tuned_parameters

{0: {'estimator_args': {'preprocessing__categorical__encoder__smoothing': 5.0,
   'preprocessing__categorical__encoder__min_samples_leaf': 100,
   'over_sampler__sampling_strategy': 0.5,
   'estimator__subsample_freq': 1,
   'estimator__subsample': 1.0,
   'estimator__reg_lambda': 0.0,
   'estimator__reg_alpha': 0.0,
   'estimator__num_leaves': 100,
   'estimator__n_estimators': 50,
   'estimator__min_split_gain': 0.1,
   'estimator__min_child_weight': 0.01,
   'estimator__min_child_samples': 100,
   'estimator__max_depth': 5,
   'estimator__learning_rate': 0.15,
   'estimator__colsample_bytree': 1.0,
   'estimator__class_weight': 'balanced',
   'estimator__boosting_type': 'dart'},
  'best_tuning_score': 0.6163042508335727,
  'tuner': 'sklearn.model_selection.RandomizedSearchCV',
  'tuner_args': {'scoring': 'roc_auc',
   'verbose': 2,
   'n_iter': 10,
   'param_distributions': {'preprocessing__categorical__encoder__min_samples_leaf': [100,
     500,
     1000,
     5000,
     10000],
 

In [26]:
best_tuning_score

0.6163042508335727

# 13- fit_model_for_training

In [27]:
%load_ext autoreload 
%autoreload 2
from utils_train_model import train_model


if len(tuned_parameters) > 1:
    ValueError("Sarah: There are more than one tuned_parameters available!!!! Can't start train_model()")
fit_models = train_model(split_train_test_cal, tuned_parameters[0], features, splitting_params, target_col_name, estimator)
fit_models


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
data.shape=(5896, 76)
len(features_per_iteration)=65
type(features_per_iteration)=<class 'list'>
  weight_samples=False, 
  splits_to_fit=TRAIN, 
  estimator_name=None, 
  estimator_args={'preprocessing__categorical__encoder__smoothing': 5.0, 'preprocessing__categorical__encoder__min_samples_leaf': 100, 'over_sampler__sampling_strategy': 0.5, 'estimator__subsample_freq': 1, 'estimator__subsample': 1.0, 'estimator__reg_lambda': 0.0, 'estimator__reg_alpha': 0.0, 'estimator__num_leaves': 100, 'estimator__n_estimators': 50, 'estimator__min_split_gain': 0.1, 'estimator__min_child_weight': 0.01, 'estimator__min_child_samples': 100, 'estimator__max_depth': 5, 'estimator__learning_rate': 0.15, 'estimator__colsample_bytree': 1.0, 'estimator__class_weight': 'balanced', 'estimator__boosting_type': 'dart'}
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 

22222

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


{0: {'feature_col_names': ['rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg',
   'dmgrphc_b_zip_code_cd',
   'rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg',
   'cust_prd_hld_d_fixedbroadband_active_30_to_60_days_avg_val',
   'cust_prd_hld_d_tvchannelpackage_closed_0_to_30_days_avg_val',
   'ci_d_variation_sum_total_num_of_interactions_last_30_to_60_days_cnt',
   'ci_d_total_num_of_interactions_weekend_last_0_to_30_cnt',
   'ci_d_sum_total_compl_last_0_to_30_days_cnt',
   'cust_prd_hld_d_mobilevoicesubscription_closed_val',
   'cust_prd_hld_d_vas_closed_0_to_30_days_avg_val',
   'cust_prd_hld_d_postpaid_tvchannelpackage_cnt',
   'cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val',
   'ci_d_sum_compl_fraga_last_0_to_60_days_cnt',
   'cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val',
   'cust_prd_hld_d_fixedbroadband_active_val',
   'cust_prd_hld_d_fixedbroadband_active_0_to_30_days_avg_val',
   'cust_prd_hld_d_vas_ac

In [28]:
# node(
#     func=train_model,
#     inputs={
#         "data": f"{catalog_str}.split_train_test_cal",
#         "estimator": f"{catalog_str}.estimator",
#         "model_params": f"{catalog_str}.tuned_parameters",
#         "features": f"{model_name}.in_memory.features",
#         "splitting_params": f"{param_str}.splitting_params",
#         "target_col_name": f"{param_str}.target",
#     },
#     outputs=f"{catalog_str}.fit_models",
#     name=f"fit_model_for_training",
#     tags=["training: model"],
# )

In [29]:
len(fit_models)

1

# 14- calibrate classifires

In [30]:
from imblearn.pipeline import Pipeline as imblearn_pipeline
from sklearn.pipeline import Pipeline as sklearn_pipeline
from sklearn.calibration import CalibratedClassifierCV

def calibrate_classifiers(data, trained_models, splitting_params, calibration_params, target_col_name):
# def calibrate_classifiers(data: pd.DataFrame, trained_models: Mapping[str, Any], splitting_params: Mapping[str, Any], calibration_params: Optional[Mapping[str, Any]] = None,) -> Mapping[str, Any]:
    """Calibrates prefit sklearn compatible models using `CalibratedClassifierCV`.

    `splitting_params` expects the following key value pairs:
        `iteration_col_name`: Name of column in which the fold id is to be saved
        `split_col_name`: Name of column in which the TRAIN/TEST split is saved

    `calibration_params` may be used with following optional key value pairs:
        `method`: (Optional) Calibration method, `sigmoid` or `isotonic`. Default is
            `sigmoid`
        `calibration_split_name`: (Optional) Split on which model will be calibrated,
            default is `CAL`

    Args:
        data: A dataframe containing features, targets, fold and split information
        trained_models: Prefit sklearn compatible classifiers
        splitting_params: Describe where fold and splitting information are located
        calibration_params: Describe which method and data to use

    Returns:
        A dictionary with the fold id as key and the model calibrated on that fold
            as value
    """
    calibration_params = {} if calibration_params is None else calibration_params
    iteration_col_name = splitting_params.get("iteration_col_name", "iteration_id")
    split_col_name = splitting_params.get("split_col_name", "split")

    calibration_method = calibration_params.get("method", "sigmoid")
    calibration_split_name = calibration_params.get("calibration_split_name", "CAL")
#     print(f"  calibration_params={calibration_params}, \n  iteration_col_name={iteration_col_name}, \n  split_col_name={split_col_name}, \n  calibration_method={calibration_method}, \n  calibration_split_name={calibration_split_name}")

    calibrated_model = data.groupby(iteration_col_name).apply(
                                _calibrate_models_per_fold,
                                trained_models,
                                calibration_method,
                                calibration_split_name,
                                split_col_name,
                                ).to_dict()
#     print(f"calibrated_model={calibrated_model}")
    for k in calibrated_model:
#         print(f"\t k = {k}")
        trained_models[k]['calibrated_model'] = calibrated_model[k]
    print(f"len(trained_models) = {len(trained_models)}")
    print(f"trained_models.keys = {trained_models.keys()}")
    return trained_models
    
#     return (
#         data.groupby(iteration_col_name)
#         .apply(
#             _calibrate_models_per_fold,
#             trained_models,
#             calibration_method,
#             calibration_split_name,
#             split_col_name,
#         )
#         .to_dict()
#     )


def _calibrate_models_per_fold(partition, trained_models, calibration_method, calibration_split_name, split_col_name):
# def _calibrate_models_per_fold(partition: pd.DataFrame, trained_models: Mapping[str, Any], calibration_method: AnyStr, calibration_split_name: AnyStr, split_col_name: AnyStr,) -> Any:
    """Calibrates a single pretrained model using `CalibratedClassifierCV`."""
    print("\n\n")
#     print(f"calibration_method={calibration_method}")
    iteration_id = partition.name
    prefit_model = trained_models[iteration_id]
#     print(f"iteration_id={iteration_id}, prefit_model={prefit_model}")

#     calibrated_model = prefit_model.calibrate(
#         partition, calibration_method, calibration_split_name, split_col_name
#     )
    
    # calibrate method from ModelContainer starts!!!! 
    fitted_model = prefit_model.get('fitted_model')
    feature_col_names = prefit_model.get("feature_col_names")
    fitted_model = prefit_model.get("fitted_model")
#     print(f"fitted_model = {fitted_model}")
#     print(f"feature_col_names = {feature_col_names}")
    _check_is_fitted(fitted_model)
    
    index_cal = partition[split_col_name] == calibration_split_name
    data_cal = partition.loc[index_cal].copy()
    X_train = data_cal[feature_col_names]
    y_train = data_cal[target_col_name]
    y_train = y_train.astype(int)
#     print(f"\n index_cal.shape = {index_cal.shape}")
#     print(f"data_cal.shape = {data_cal.shape}")
#     print(f"X_train.shape = {X_train.shape}")
#     print(f"y_train.shape = {y_train.shape}")

    if isinstance(fitted_model, (imblearn_pipeline, sklearn_pipeline)):
        print("11111111")
        pipeline = fitted_model
        estimator_tuple = pipeline.steps[-1]
#         print(f"estimator_tuple={estimator_tuple}")
        preprocessor =  pipeline.steps[0][1]
        calibrator = CalibratedClassifierCV(base_estimator=estimator_tuple[1], cv="prefit", method=calibration_method)
        calibrated_model = calibrator.fit(preprocessor.transform(X_train), y_train)
        pipeline.steps[-1] = (estimator_tuple[0], calibrator)
    else:
        print("222222222222")
        calibrator = CalibratedClassifierCV(base_estimator=fitted_model, method=calibration_method, cv="prefit")

        calibrated_model = calibrator.fit(X_train, y_train)

    return calibrated_model
#     print(f"calibrated_model = {calibrated_model}")
#     print(f"len(trained_models[{iteration_id}]) = {len(trained_models[iteration_id])}, trained_models[{iteration_id}] = {trained_models[iteration_id].keys()}")
#     trained_models[iteration_id]['calibrated_model'] = calibrated_model
#     print("********************")
#     print(f"\n\nlen(trained_models[{iteration_id}]) = {len(trained_models[iteration_id])}, trained_models[{iteration_id}] = {trained_models[iteration_id].keys()}")    
#     print("---------------------------------------")
#     return trained_models


def _check_is_fitted(fitted_model):
    """Checks whether the model has been fitted already."""
    if fitted_model is None:
        raise NotFittedError("This estimator has not been fitted yet.")

In [31]:
print(f"Before calibrate_classifiers(): \nlen(fit_models) = {len(fit_models)}, keys={len(fit_models[0].keys())}, and keys are: \n {fit_models[0].keys()}")

Before calibrate_classifiers(): 
len(fit_models) = 1, keys=11, and keys are: 
 dict_keys(['feature_col_names', 'estimator_name', 'estimator_args', 'splits_to_fit', 'weight_samples', 'model', 'target_col_name', 'weights', 'calibration_method', 'feature_importances_', 'fitted_model'])


In [32]:
calibration_params = {'method': 'sigmoid', 'calibration_split_name': 'CAL'}

calibrated_model = calibrate_classifiers(split_train_test_cal, fit_models, splitting_params, calibration_params, target)





11111111
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
len(trained_models) = 1
trained_models.keys = dict_keys([0])


In [33]:
print(f"After calibrate_classifiers(): \nlen(fit_models) = {len(fit_models)}, keys={len(fit_models[0].keys())}, and keys are: \n {fit_models[0].keys()}")

After calibrate_classifiers(): 
len(fit_models) = 1, keys=12, and keys are: 
 dict_keys(['feature_col_names', 'estimator_name', 'estimator_args', 'splits_to_fit', 'weight_samples', 'model', 'target_col_name', 'weights', 'calibration_method', 'feature_importances_', 'fitted_model', 'calibrated_model'])


In [34]:
calibrated_model

{0: {'feature_col_names': ['rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg',
   'dmgrphc_b_zip_code_cd',
   'rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg',
   'cust_prd_hld_d_fixedbroadband_active_30_to_60_days_avg_val',
   'cust_prd_hld_d_tvchannelpackage_closed_0_to_30_days_avg_val',
   'ci_d_variation_sum_total_num_of_interactions_last_30_to_60_days_cnt',
   'ci_d_total_num_of_interactions_weekend_last_0_to_30_cnt',
   'ci_d_sum_total_compl_last_0_to_30_days_cnt',
   'cust_prd_hld_d_mobilevoicesubscription_closed_val',
   'cust_prd_hld_d_vas_closed_0_to_30_days_avg_val',
   'cust_prd_hld_d_postpaid_tvchannelpackage_cnt',
   'cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val',
   'ci_d_sum_compl_fraga_last_0_to_60_days_cnt',
   'cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val',
   'cust_prd_hld_d_fixedbroadband_active_val',
   'cust_prd_hld_d_fixedbroadband_active_0_to_30_days_avg_val',
   'cust_prd_hld_d_vas_ac

In [35]:
# node(
#     func=calibrate_classifiers,
#     inputs={
#         "data": f"{catalog_str}.split_train_test_cal",
#         "trained_models": f"{catalog_str}.fit_models",
#         "splitting_params": f"{param_str}.splitting_params",
#         "calibration_params": f"{param_str}.calibration_params",
#     },
#     outputs=f"{catalog_str}.calibrated_model",
#     name=f"calibrate_model_for_training",
#     tags=["training: model"],
# )

# 16- get_model_predictions

# <i> Note: calibrated_model is the same as fited_models


In [36]:
def get_model_predictions(data, trained_models, splitting_params, inference_params, target_col_name):
# def get_model_predictions(data: pd.DataFrame, trained_models: Mapping[str, Any], splitting_params: Mapping[str, Any], inference_params: Mapping[str, Any] = None,) -> pd.DataFrame:
    """Makes predictions for each fold given a dictionary of trained models.

    Predictions are saved into a new column. If the trained model implements a
    `predict_proba` method and is a binary classifier, then the prediction
    score for the second class is stored in an additional new columns with the name
    `target` + `prediction_proba_suffix`. The `prediction_proba_suffix` can be provided
    in the `inference_params`.

    `inference_params` expects the following key value pairs:
        `prediction_suffix`: (Optional) Suffix appended to target to form prediction
            column name. Default is `_pred`
        `prediction_proba_suffix`: (Optional) Suffix appended to target to form
            prediction score name. Default is `_pred_score`

    `splitting_params` expects the following key value pairs:
        `iteration_col_name`: Name of column in which the fold id is to be saved

    Args:
        data: Data containing fold id and features
        trained_models: A dictionary with fold ids as keys and trained models as values
        splitting_params: A dictionary describing how splitting is done, including
            in which column the iteration id resides
        inference_params: (Optional) Parameters including the suffix to use
            for predictions

    Returns:
        A new dataframe with the predictions as a new column, and the prediction scores
        if a `predict_proba` method is available and it's a binary classifier.
    """
    iteration_col_name = splitting_params.get("iteration_col_name", "iteration_id")

    inference_params = inference_params if inference_params else {} # {'prediction_suffix': '_pred'}

    prediction_suffix = inference_params.get("prediction_suffix", "_pred") # '_pred'
    prediction_proba_suffix = inference_params.get("prediction_proba_suffix", "_pred_score") # '_pred_score'
    

    print(f"  iteration_col_name={iteration_col_name}, \n  inference_params={inference_params}\n  prediction_suffix={prediction_suffix} \n  prediction_proba_suffix={prediction_proba_suffix} \n")
    
    return data.groupby(iteration_col_name).apply(
        _make_predictions_per_fold,
        trained_models,
        prediction_suffix,
        prediction_proba_suffix,
    )


def _make_predictions_per_fold(partition, trained_models, prediction_suffix, prediction_proba_suffix):
# def _make_predictions_per_fold(partition: pd.DataFrame, trained_models: Mapping[str, Any], prediction_suffix: str, prediction_proba_suffix: str,):
    """Makes predictions for one fold.

    If the model implements a `predict_proba` method and is a binary
    classifier, then the prediction score is calculated, too.
    """
#     print("\n\n****************")
#     print(f"trained_models: {trained_models}")
    iteration_id = partition.name # 0
#     print(f"iteration_id={iteration_id}")
    model = trained_models[iteration_id]
#     print(f"model.keys = {model.keys()}")

    target_col_name = model.get('target_col_name')
#     print(f"\ntarget_col_name={target_col_name}")
    
    pred_col_name = target_col_name + prediction_suffix
    pred_score_col_name = (
        target_col_name + prediction_proba_suffix
        if hasattr(model.get("fitted_model"), "predict_proba")
        else None
    ) # 'tgt_xsell_cust_voice_to_fixed_pred_score'

#     print(f"pred_col_name={pred_col_name}, \npred_score_col_name={pred_score_col_name}")
    
    # predict model from ModelContainer starts!!! 
#     partition = model.predict(partition, pred_col_name, pred_score_col_name)
#         _check_is_fitted(self.fitted_model)

    if (pred_col_name is None) and (pred_score_col_name is None):
        raise TypeError(
            "Expected pred_col_name or pred_score_col_name, neither was given."
        )

    if pred_col_name is not None:
#         print("1111111111")
#         print(f"len feature_col_names = {len(model.get('feature_col_names'))}")
        prediction = model.get("fitted_model").predict(partition[model.get("feature_col_names")]) # len(self.feature_col_names) = 160, rediction.shape = (326297,)
        partition[pred_col_name] = prediction # (326297, 166)
#         print(f"prediction = {prediction}")

    if pred_score_col_name is not None:

        if not hasattr(model.get("fitted_model"), "predict_proba"):
#             print("22222222222")
            raise AttributeError(
                f"A pred_score_col_name was passed, but {type(model.get('fitted_model'))} "
                + "has no attribute predict_proba. This indicates that the model "
                + "is not a classifier and can not predict a score. "
                + "Remove the pred_score_col_name argument or change "
                + "the model to a classifier."
            )

        prediction_score = model.get("fitted_model").predict_proba(partition[model.get("feature_col_names")]) # (326297, 2)
#         print(f"\nprediction_score.shape = {prediction_score.shape}")
        if len(model.get("fitted_model").classes_) > 2:
#             print("3333333333333333333")
            class_score_names = [
                pred_score_col_name + "_" + str(category)
                for category in model.get("fitted_model").classes_
            ]
            for (i, class_score_name) in enumerate(class_score_names):
                partition[class_score_name] = prediction_score[:, i]

        else:
#             print("44444444444444444444")
            partition[pred_score_col_name] = prediction_score[:, -1] # goes to else!

#     print(f"partition.shape={partition.shape}")
    return partition

In [37]:
calibrated_model[0].keys()

dict_keys(['feature_col_names', 'estimator_name', 'estimator_args', 'splits_to_fit', 'weight_samples', 'model', 'target_col_name', 'weights', 'calibration_method', 'feature_importances_', 'fitted_model', 'calibrated_model'])

In [38]:
inference_params = {'prediction_suffix': '_pred'}

# calibrated_model is the same as fited_models

predictions = get_model_predictions(split_train_test_cal, calibrated_model, splitting_params, inference_params, target)
predictions

  iteration_col_name=iteration_id, 
  inference_params={'prediction_suffix': '_pred'}
  prediction_suffix=_pred 
  prediction_proba_suffix=_pred_score 

 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 


,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0,iteration_id,split,tgt_xsell_cust_voice_to_fixed_pred,tgt_xsell_cust_voice_to_fixed_pred_score
0,100254,2021-05-26,0,42,68,U,East Jacquelineshire,Liechtenstein,0,Bad,...,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN,0,0.130084
1,100249,2021-06-25,0,24,91,U,South Marissa,Qatar,3,Bad,...,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed,0,CAL,0,0.209863
2,100249,2021-06-25,0,24,91,U,South Marissa,Qatar,3,Bad,...,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN,0,0.209863
3,100249,2021-06-25,0,24,91,U,South Marissa,Qatar,3,Bad,...,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN,0,0.209863
4,100307,2021-08-01,0,52,41,M,Holderburgh,Seychelles,4,Bad,...,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN,0,0.230189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891,10060,2021-06-11,0,40,64,F,Cruzview,Saint Martin,1,Bad,...,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN,0,0.326854
5892,100642,2021-06-21,0,50,46,M,Rollinsville,Germany,2,Bad,...,0,0,272.358113,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN,0,0.102301
5893,100899,2021-07-14,0,64,44,U,Melissashire,British Indian Ocean Territory (Chagos Archipe...,0,Good,...,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN,0,0.121276
5894,100899,2021-07-14,0,64,44,U,Melissashire,British Indian Ocean Territory (Chagos Archipe...,0,Good,...,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN,0,0.121276


In [39]:
# node(
#     func=get_model_predictions,
#     inputs={
#         "data": f"{catalog_str}.split_train_test_cal",
#         "trained_models": f"{catalog_str}.calibrated_model",
#         "splitting_params": f"{param_str}.splitting_params",
#         "inference_params": f"{param_str}.inference_params",
#     },
#     outputs=f"{catalog_str}.predictions",
#     name=f"prediction_for_training",
#     tags=["training: predictions"],
# )

# 17- get_model_performance

In [40]:
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, f1_score, precision_score, recall_score, confusion_matrix


# pylint: disable=too-many-locals
def get_model_performance(data, metrics_params, splitting_params, target_col_name, inference_params):
# def get_model_performance(data: pd.DataFrame, metrics_params: Mapping[str, Any], splitting_params: Mapping[str, Any], target_col_name: str, inference_params: Mapping[str, Any] = None,) -> pd.DataFrame:
    """Get model performance.

    Saves train/test metrics specified in the parameters file, for individual
    folds and overall to PAI and returns them in a dictionary.

    `metrics_params` expects the following key value pairs:
        `metrics`: sklearn performance metrics
        `tracker`: tracker name for logging metrics

    `splitting_params` expects the following key value pairs:
        `iteration_col_name`: Name of column in which the fold id is to be saved
        `split_col_name`: name of column in which train/test split is given

    Args:
        data: Dataframe containing features, targets, predictions, fold and
            split information
        metrics_params: contains information about metrics to compute and the tracker
            for logging the metrics.
        splitting_params: Describes where fold and splitting info are located
        target_col_name: Name of target column
        inference_params: (Opt) Contains the prediction suffixes.

    Returns:
        Dictionary containing train and test metrics for each fold and for
        overall data.

    """

    iteration_col_name = splitting_params.get("iteration_col_name", "iteration_id")
    split_col_name = splitting_params.get("split_col_name", "split")

    inference_params = inference_params if inference_params else {}

    prediction_suffix = inference_params.get("prediction_suffix", "_pred")
    prediction_proba_suffix = inference_params.get(
        "prediction_proba_suffix", "_pred_score"
    )

    prediction_col_name = target_col_name + prediction_suffix
    pred_score_col_name = target_col_name + prediction_proba_suffix

    model_metrics = {}
    scorers = []

    metric_params = metrics_params["metrics"]
    tracker_name = metrics_params.get("tracker")
    
#     print(f"\titeration_col_name= {iteration_col_name}, \n\tsplit_col_name = {split_col_name}, \n\tinference_params={inference_params},  \n\tprediction_suffix={prediction_suffix}, \n\tprediction_proba_suffix={prediction_proba_suffix} , \n\tpred_score_col_name={pred_score_col_name}, \n\tmetric_params={metric_params}, \n\ttracker_name={tracker_name}")
    
    """Computes the value of the metric for train and test sets."""
    train_index = data[split_col_name] == "TRAIN"
    test_index = data[split_col_name] == "TEST"

    y_train = data.loc[train_index, target_col_name]
    y_pred_train = data.loc[train_index, prediction_col_name]

    y_test = data.loc[test_index, target_col_name]
    y_pred_test = data.loc[test_index, prediction_col_name]
    

    scorer_name = 'roc_auc_score'
#     print(f"\n--------------------- {scorer_name}")
#     roc_auc_score(y_true, y_score, *, average='macro', sample_weight=None, max_fpr=None, multi_class='raise', labels=None)
    train_metric_value_roc = roc_auc_score(y_train.values.astype(int), y_pred_train.values.astype(int))
    test_metric_value_roc = roc_auc_score(y_test.values.astype(int), y_pred_test.values.astype(int))
#     print(f"train_metric_value={train_metric_value_roc}, \t test_metrics_value={test_metric_value_roc}")
    scorers.append(f"train_{scorer_name}")
    scorers.append(f"test_{scorer_name}")
    model_metrics = _append_float_metric(model_metrics,train_metric_value_roc,test_metric_value_roc,target_col_name,"overall",scorer_name)
    
    
    scorer_name = 'balanced_accuracy_score'
#     print(f"\n--------------------- {scorer_name}")
#     balanced_accuracy_score(y_true, y_pred, *, sample_weight=None, adjusted=False)
    train_metric_value_balanced = balanced_accuracy_score(y_train.values.astype(int), y_pred_train.values.astype(int))
    test_metric_value_balanced = balanced_accuracy_score(y_test.values.astype(int), y_pred_test.values.astype(int))
#     print(f"train_metric_value={train_metric_value_balanced}, \t test_metric_value={test_metric_value_balanced}")
    scorers.append(f"train_{scorer_name}")
    scorers.append(f"test_{scorer_name}")
    model_metrics = _append_float_metric(model_metrics,train_metric_value_balanced,test_metric_value_balanced,target_col_name,"overall",scorer_name)
  

    scorer_name = 'f1_score'
#     print(f"\n--------------------- {scorer_name}")
#     f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
    train_metric_value_f1 = f1_score(y_train.values.astype(int), y_pred_train.values.astype(int))
    test_metric_value_f1 = f1_score(y_test.values.astype(int), y_pred_test.values.astype(int))
#     print(f"train_metric_value={train_metric_value_f1}, \t test_metric_value={test_metric_value_f1}")
    scorers.append(f"train_{scorer_name}")
    scorers.append(f"test_{scorer_name}")
    model_metrics = _append_float_metric(model_metrics,train_metric_value_balanced,test_metric_value_balanced,target_col_name,"overall",scorer_name)
    
    
    scorer_name = 'precision_score'
#     print(f"\n--------------------- {scorer_name}")
#     precision_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
    train_metric_value_precision_score = precision_score(y_train.values.astype(int), y_pred_train.values.astype(int))
    test_metric_value_precision_score = precision_score(y_test.values.astype(int), y_pred_test.values.astype(int))
#     print(f"train_metric_value={train_metric_value_precision_score}, \t test_metric_value={test_metric_value_precision_score}")
    scorers.append(f"train_{scorer_name}")
    scorers.append(f"test_{scorer_name}")
    model_metrics = _append_float_metric(model_metrics,train_metric_value_balanced,test_metric_value_balanced,target_col_name,"overall",scorer_name)

    
    scorer_name = 'recall_score'
#     print(f"\n--------------------- {scorer_name}")
#     recall_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
    train_metric_value_recall_score = recall_score(y_train.values.astype(int), y_pred_train.values.astype(int))
    test_metric_value_recall_score = recall_score(y_test.values.astype(int), y_pred_test.values.astype(int))
#     print(f"train_metric_value={train_metric_value_recall_score}, \t test_metric_value={test_metric_value_recall_score}")
    scorers.append(f"train_{scorer_name}")
    scorers.append(f"test_{scorer_name}")
    model_metrics = _append_float_metric(model_metrics,train_metric_value_balanced,test_metric_value_balanced,target_col_name,"overall",scorer_name)
    
    
    scorer_name = 'confusion_matrix'
#     print(f"\n--------------------- {scorer_name}")
#     confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)
    train_metric_value_confusion_matrix = confusion_matrix(y_train.values.astype(int), y_pred_train.values.astype(int))
    test_metric_value_confusion_matrix = confusion_matrix(y_test.values.astype(int), y_pred_test.values.astype(int))
#     print(f"train_metric_value={train_metric_value_confusion_matrix}, \t test_metric_value={test_metric_value_confusion_matrix}")
    score_names = ["true_negative","false_positive","false_negative","true_positive",]
    for name in score_names:
        scorers.append(f"train_{name}")
        scorers.append(f"test_{name}")
    model_metrics = _append_confusion_matrix(model_metrics,train_metric_value_confusion_matrix, test_metric_value_confusion_matrix, target_col_name,"overall",)

    print(f"\n\nmodel_metrics = {model_metrics['overall_tgt_xsell_cust_voice_to_fixed']}")
    print(f"\nscorers = {scorers}")
    print(f"\n\nmodel_metrics = {len(model_metrics['overall_tgt_xsell_cust_voice_to_fixed'])}")
    print(f"\nscorers = {len(scorers)}")    

#     result = (pd.DataFrame.from_dict(model_metrics, orient="index", columns=scorers)
#               .reset_index()
#               .rename(columns={"index": "group"}))
# #     tracking.get_tracker(tracker_name).log_artifacts({"results": result})
#     return result
    return "code is not complete!! "


def _append_float_metric(metrics_dict, train_metric_value, test_metric_value, target_col_name, key_name, scorer_name,):
    metrics_dict = _add_value_to_dict(
        train_metric_value, f"{key_name}_{target_col_name}", metrics_dict
    )
    metrics_dict[f"{key_name}_{target_col_name}"].append(test_metric_value)

    return metrics_dict


def _add_value_to_dict(value, key_name, metric_dict):
    """Checks if a key is in the dictionary and adds the required value."""
    if key_name in metric_dict:
        metric_dict[key_name].append(value)
    else:
        metric_dict[key_name] = [value]

    return metric_dict

def _append_confusion_matrix( metrics_dict, train_metric_value, test_metric_value, target_col_name, key_name):
    if len(train_metric_value) == 2:
    # if train_metric_value.shape[0] == 2:
        i =0 
        for train_value, test_value in zip(
            # train_metric_value.ravel(), test_metric_value.ravel()
            train_metric_value, test_metric_value
        ):
#             print(f"i={i}")
#             i = i + 1
#             print("train_value", train_value)
#             print("test_value", test_value)
            metrics_dict = _add_value_to_dict(
                train_value, f"{key_name}_{target_col_name}", metrics_dict
            )
            metrics_dict[f"{key_name}_{target_col_name}"].append(test_value)

    else:
        raise ValueError(
            "Performance table doesn't support confusion matrices for multi-class "
            "classification"
        )

    return metrics_dict

In [41]:
metrics_params = {'tracker': 'exp', 
                  'metrics': ['sklearn.metrics.roc_auc_score', 
                              'sklearn.metrics.balanced_accuracy_score', 
                              'sklearn.metrics.f1_score', 
                              'sklearn.metrics.precision_score', 
                              'sklearn.metrics.recall_score', 
                              'sklearn.metrics.confusion_matrix']}
inference_params = {'prediction_suffix': '_pred'}

performance = get_model_performance(predictions, metrics_params, splitting_params, target, inference_params)
performance



model_metrics = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, array([3412,    0]), array([1024,    0]), array([776,   0]), array([218,   0])]

scorers = ['train_roc_auc_score', 'test_roc_auc_score', 'train_balanced_accuracy_score', 'test_balanced_accuracy_score', 'train_f1_score', 'test_f1_score', 'train_precision_score', 'test_precision_score', 'train_recall_score', 'test_recall_score', 'train_true_negative', 'test_true_negative', 'train_false_positive', 'test_false_positive', 'train_false_negative', 'test_false_negative', 'train_true_positive', 'test_true_positive']


model_metrics = 14

scorers = 18


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'code is not complete!! '

In [42]:
# node(
#     func=get_model_performance,
#     inputs={
#         "data": f"{catalog_str}.predictions",
#         "metrics_params": f"{param_str}.metrics_params",
#         "splitting_params": f"{param_str}.splitting_params",
#         "target_col_name": f"{param_str}.target",
#         "inference_params": f"{param_str}.inference_params",
#     },
#     outputs=f"{catalog_str}.performance",
#     name=f"performance_for_training",
#     tags=["training: performance"],
# )

# Inference

In [61]:
import re

def to_pandas(data, pandas_params, spine_params, target_col, data_dictionary, features, ref_date):
# def to_pandas(data: DataFrame, pandas_params: Dict[str, Any], spine_params: Dict[str, Any], target_params: Dict[str, Any], data_dictionary: Optional[DataDict] = None, features: List[str] = None, ref_date: Optional[str] = None,) -> pd.DataFrame:
    ref_period = pandas_params.get("ref_period")
    include_target = pandas_params.get("include_target")
    key_col = to_list(spine_params.get('keys'))
    date_col = spine_params.get('date_column')
#     target_col = target_params.get('target_variable_column')

    if ref_date and ref_period:
        raise AttributeError(
            "Cannot specify both ref_date and ref_period at"
            "the same time. Specify only one argument."
        )
        
    print(f"\tref_period={ref_period}, \n\tinclude_target={include_target}, \n\tkey_col={key_col}, \n\tdate_col={date_col}, \n\ttarget_col={target_col}, \n\tdata_dictionary={data_dictionary}")
    print(f"len(features)= {len(features)}")
    
    feat_col, target_switch_col = [None] * 2
    
    if data_dictionary or features:
        if data_dictionary:
            feat_col = data_dictionary.get_features()
            target_switch_col = data_dictionary.get_target_switch_variable()
        elif features:
            feat_col = features

        export_cols = []
        if date_col:
            export_cols += [date_col]
        if key_col:
            export_cols += key_col
        export_cols += feat_col
        if include_target and target_col:
            export_cols += [target_col]

        cols_diff = set(export_cols) - set(data.columns)
        if len(cols_diff) != 0:
            # logger.warning(f"The following columns are not available in the DataFrame and will be ignored: {cols_diff}")
            export_cols = list(set(export_cols) - cols_diff)

        print(f"len(export_cols)={len(export_cols)}")
        print(f"Before: {data.shape}")
        data = data[export_cols]
        print(f"After: {data.shape}")        

    if ref_date and date_col:
        try:
            if not re.match(r"\d{4}-\d{2}-\d{2}", ref_date):
                raise ValueError("Date format should follow yyyy-mm-dd.")

            ref_date = pd.to_datetime(ref_date, infer_datetime_format=True).strftime("%Y-%m-%d")
            print(f"ref_date={ref_date}, type = {type(ref_date)}")
            
            data['dt'] = pd.to_datetime(data[date_col], format='%Y-%m-%d %H:%M:%S')
            data = data[data['dt'] == ref_date]
            print(f"After filter for day {ref_date} data.shape = {data.shape}")            
            data = data.drop(columns=['dt'])
            print(f"After drop filter for day {ref_date} data.shape = {data.shape}")
        except (ValueError, TypeError) as exception:
            raise exception("'ref_date' type casting failed") from exception

    # TODO: Sarah The followings are not tested!! 
    print(f"first if = {include_target and target_col}, 2nd if={include_target and target_switch_col}, 3rd if={ref_period and date_col}")
    if include_target and target_col:
        print(f"\nBefore include_target shape is {data.shape}")
        data = data[data[target_col.notnull()]]
        print(f"After include_target shape is {data.shape}")

    if include_target and target_switch_col:
        print(f"\nBefore include_target and target_switch_col shape is {data.shape}")
        # data = data.filter(~f.col(target_switch_col))
        data = data[data[target_switch_col]]
        print(f"After include_target and target_switch_col shape is {data.shape}")
                    
    if ref_period and date_col:
        print(f"\nBefore filter_ref_window shape is {data.shape}")
        # data = filter_ref_window(data, ref_period, date_col)
        print(f"After filter_ref_window shape is {data.shape}")
    
    return data


def to_list(item):
    """
    Wraps an item to a list or returns the existing list

    Args:
        item (Union[Any, List[Any]]): a list of a single object

    Returns:
        item wrapped in a list, if item is not a list. Otherwise the
        original list is returned
    """

    return [item] if not isinstance(item, list) else item

In [62]:
pandas_params = {'include_target': False}
spine_params = {'keys': ['customer_id'], 
                'date_column': 'current_dt', 
                'product_holdings_filter': {'exact_match': {'product_category': 'fixedbroadband'}}, 
                'is_deepsell': 'N'}
ref_date = '2021-06-30'
data_dictionary = None
inference_master_table = to_pandas(df, pandas_params, spine_params, target, data_dictionary, features, ref_date)
inference_master_table


	ref_period=None, 
	include_target=False, 
	key_col=['customer_id'], 
	date_col=current_dt, 
	target_col=tgt_xsell_cust_voice_to_fixed, 
	data_dictionary=None
len(features)= 65
len(export_cols)=67
Before: (40209, 74)
After: (40209, 67)
ref_date=2021-06-30, type = <class 'str'>
After filter for day 2021-06-30 data.shape = (295, 68)
After drop filter for day 2021-06-30 data.shape = (295, 67)
first if = False, 2nd if=False, 3rd if=None


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,current_dt,customer_id,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,dmgrphc_b_zip_code_cd,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,cust_prd_hld_d_fixedbroadband_active_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_closed_0_to_30_days_avg_val,ci_d_variation_sum_total_num_of_interactions_last_30_to_60_days_cnt,ci_d_total_num_of_interactions_weekend_last_0_to_30_cnt,ci_d_sum_total_compl_last_0_to_30_days_cnt,...,ci_d_sum_total_resolved_cmpl_last_0_to_30_days_val,dmgrphc_d_tenure_in_days_val,cust_prd_hld_d_tvchannelpackage_active_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_val,rev_m_eom_total_bill_amt_m1_val,cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_closed_30_to_60_days_avg_val,ci_d_variation_sum_total_resolved_cmpl_last_30_to_60_days_val,cust_prd_hld_d_vas_active_val
115,2021-06-30,100389,<NA>,23,<NA>,0.000000,0.000000,NaN,0,0,...,0,701,0.0,0,NaN,1.923077,NaN,0.0,NaN,2
116,2021-06-30,100389,<NA>,23,<NA>,0.000000,0.000000,NaN,0,0,...,0,701,0.0,0,NaN,1.923077,NaN,0.0,NaN,2
250,2021-06-30,100657,0,94,0,1.387097,0.000000,NaN,0,0,...,0,300,0.0,0,0.0,1.551724,NaN,0.0,NaN,2
365,2021-06-30,100440,<NA>,12,<NA>,1.000000,0.000000,NaN,0,0,...,0,630,0.0,0,NaN,2.400000,NaN,0.0,NaN,1
397,2021-06-30,100886,<NA>,86,<NA>,0.000000,0.000000,NaN,0,0,...,0,-23,0.0,0,NaN,1.000000,NaN,0.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39906,2021-06-30,100310,0,40,0,0.000000,0.806452,1.0,1,0,...,0,811,2.0,2,0.0,1.000000,NaN,0.0,NaN,1
39969,2021-06-30,100807,0,15,0,0.000000,0.000000,NaN,0,0,...,0,93,0.0,0,0.0,NaN,NaN,0.0,NaN,0
39992,2021-06-30,100235,<NA>,31,<NA>,0.000000,0.000000,1.0,0,0,...,0,914,1.0,1,NaN,0.000000,NaN,0.0,NaN,0
40079,2021-06-30,100783,<NA>,38,<NA>,0.000000,0.000000,NaN,0,0,...,0,123,0.0,0,NaN,0.672131,NaN,0.0,NaN,1


In [ ]:
# node(
#     func=spark_utils.to_pandas,
#     inputs={
#         "data": f"{model_name}.master.mst_tbl",
#         "pandas_params": f"{param_str}.inference_spark_to_pandas",
#         "spine_params": f"params:{model_name}.master.spine",
#         "target_params": f"params:{model_name}.master.target",
#         "features": f"{model_name}.in_memory.features_to_export",
#         "ref_date": "params:ref_date",
#     },
#     outputs=f"{model_name}.in_memory.inference_master_table",
#     name=f"master_table_to_pandas_for_inference",
#     tags=["inference: CDL to edge"],
# ),

# attach_live_iter_split_col

In [64]:
def attach_live_iter_split_col(data, splitting_params):
# def attach_live_iter_split_col(data: pd.DataFrame, splitting_params: Mapping[str, Any]) -> pd.DataFrame:
    """Adds fold and split columns to any dataframe.

    The fold is set to 0, split is set to "LIVE". This is useful for live data from
    e.g. a UI.

    `splitting_params` expects the following keys:
        `iteration_col_name`: Name of column in which the fold id is to be saved
        `split_col_name`: Name of column in which the TRAIN/TEST split is saved

    Args:
        data: Model input data containing spine, features and target
        splitting_params: Dictionary with splitting configuration

    Returns:
        A new pandas dataframe with a fold and split column attached
    """
    iteration_col_name = splitting_params.get("iteration_col_name", "iteration_id")
    split_col_name = splitting_params.get("split_col_name", "split")

    data[iteration_col_name] = 0
    data[split_col_name] = "LIVE"

    return data

In [65]:
# splitting_params = {'splitter': 'customerone.pipelines.telco.test_train_splitter.OutOfTimeSplitter', 
#                     'splitter_args': {'n_splits': 1, 
#                                       'test_size': 0.2, 
#                                       'date_col': 'current_dt', 
#                                       'verbose': 1, 
#                                       'random_state': 42}, 
#                     'group_col_name': 'customer_id', 
#                     'iteration_col_name': 'iteration_id', 
#                     'split_col_name': 'split'}

inference_master_table_live = attach_live_iter_split_col(inference_master_table, splitting_params)
inference_master_table_live

,current_dt,customer_id,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,dmgrphc_b_zip_code_cd,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,cust_prd_hld_d_fixedbroadband_active_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_closed_0_to_30_days_avg_val,ci_d_variation_sum_total_num_of_interactions_last_30_to_60_days_cnt,ci_d_total_num_of_interactions_weekend_last_0_to_30_cnt,ci_d_sum_total_compl_last_0_to_30_days_cnt,...,cust_prd_hld_d_tvchannelpackage_active_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_val,rev_m_eom_total_bill_amt_m1_val,cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_closed_30_to_60_days_avg_val,ci_d_variation_sum_total_resolved_cmpl_last_30_to_60_days_val,cust_prd_hld_d_vas_active_val,iteration_id,split
115,2021-06-30,100389,<NA>,23,<NA>,0.000000,0.000000,NaN,0,0,...,0.0,0,NaN,1.923077,NaN,0.0,NaN,2,0,LIVE
116,2021-06-30,100389,<NA>,23,<NA>,0.000000,0.000000,NaN,0,0,...,0.0,0,NaN,1.923077,NaN,0.0,NaN,2,0,LIVE
250,2021-06-30,100657,0,94,0,1.387097,0.000000,NaN,0,0,...,0.0,0,0.0,1.551724,NaN,0.0,NaN,2,0,LIVE
365,2021-06-30,100440,<NA>,12,<NA>,1.000000,0.000000,NaN,0,0,...,0.0,0,NaN,2.400000,NaN,0.0,NaN,1,0,LIVE
397,2021-06-30,100886,<NA>,86,<NA>,0.000000,0.000000,NaN,0,0,...,0.0,0,NaN,1.000000,NaN,0.0,NaN,1,0,LIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39906,2021-06-30,100310,0,40,0,0.000000,0.806452,1.0,1,0,...,2.0,2,0.0,1.000000,NaN,0.0,NaN,1,0,LIVE
39969,2021-06-30,100807,0,15,0,0.000000,0.000000,NaN,0,0,...,0.0,0,0.0,NaN,NaN,0.0,NaN,0,0,LIVE
39992,2021-06-30,100235,<NA>,31,<NA>,0.000000,0.000000,1.0,0,0,...,1.0,1,NaN,0.000000,NaN,0.0,NaN,0,0,LIVE
40079,2021-06-30,100783,<NA>,38,<NA>,0.000000,0.000000,NaN,0,0,...,0.0,0,NaN,0.672131,NaN,0.0,NaN,1,0,LIVE


In [ ]:
# node(
#     func=attach_live_iter_split_col,
#     inputs={
#         "data": f"{model_name}.in_memory.inference_master_table",
#         "splitting_params": f"{param_str}.splitting_params",
#     },
#     outputs=f"{model_name}.inference.inference_master_table",
#     name=f"add_iteration_id",
#     tags=["inference: CDL to edge"],
# )

# inference predictions 

In [67]:
# predictions = get_model_predictions(split_train_test_cal, calibrated_model, splitting_params, inference_params, target)


get_model_predictions(inference_master_table_live, calibrated_model, splitting_params, inference_params, target)


  iteration_col_name=iteration_id, 
  inference_params={'prediction_suffix': '_pred'}
  prediction_suffix=_pred 
  prediction_proba_suffix=_pred_score 

 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 
 ****** PandasSimpleImputer > transform ******** 


,current_dt,customer_id,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,dmgrphc_b_zip_code_cd,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,cust_prd_hld_d_fixedbroadband_active_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_closed_0_to_30_days_avg_val,ci_d_variation_sum_total_num_of_interactions_last_30_to_60_days_cnt,ci_d_total_num_of_interactions_weekend_last_0_to_30_cnt,ci_d_sum_total_compl_last_0_to_30_days_cnt,...,rev_m_eom_total_bill_amt_m1_val,cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_closed_30_to_60_days_avg_val,ci_d_variation_sum_total_resolved_cmpl_last_30_to_60_days_val,cust_prd_hld_d_vas_active_val,iteration_id,split,tgt_xsell_cust_voice_to_fixed_pred,tgt_xsell_cust_voice_to_fixed_pred_score
115,2021-06-30,100389,<NA>,23,<NA>,0.000000,0.000000,NaN,0,0,...,NaN,1.923077,NaN,0.0,NaN,2,0,LIVE,0,0.070897
116,2021-06-30,100389,<NA>,23,<NA>,0.000000,0.000000,NaN,0,0,...,NaN,1.923077,NaN,0.0,NaN,2,0,LIVE,0,0.070897
250,2021-06-30,100657,0,94,0,1.387097,0.000000,NaN,0,0,...,0.0,1.551724,NaN,0.0,NaN,2,0,LIVE,0,0.201995
365,2021-06-30,100440,<NA>,12,<NA>,1.000000,0.000000,NaN,0,0,...,NaN,2.400000,NaN,0.0,NaN,1,0,LIVE,0,0.169933
397,2021-06-30,100886,<NA>,86,<NA>,0.000000,0.000000,NaN,0,0,...,NaN,1.000000,NaN,0.0,NaN,1,0,LIVE,0,0.071838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39906,2021-06-30,100310,0,40,0,0.000000,0.806452,1.0,1,0,...,0.0,1.000000,NaN,0.0,NaN,1,0,LIVE,0,0.182445
39969,2021-06-30,100807,0,15,0,0.000000,0.000000,NaN,0,0,...,0.0,NaN,NaN,0.0,NaN,0,0,LIVE,0,0.250360
39992,2021-06-30,100235,<NA>,31,<NA>,0.000000,0.000000,1.0,0,0,...,NaN,0.000000,NaN,0.0,NaN,0,0,LIVE,0,0.209154
40079,2021-06-30,100783,<NA>,38,<NA>,0.000000,0.000000,NaN,0,0,...,NaN,0.672131,NaN,0.0,NaN,1,0,LIVE,0,0.184948


In [ ]:
                node(
                    func=get_model_predictions,
                    inputs={
                        "data": f"{model_name}.inference.inference_master_table",
                        "trained_models": f"{model_name}.in_memory.inf_model",
                        "splitting_params": f"{param_str}.splitting_params",
                        "inference_params": f"{param_str}.inference_params",
                    },
                    outputs=f"{model_name}.in_memory.inf_predictions",
                    name=f"prediction",
                    tags=["inference: predictions"],
                ),
                node(
                    func=add_run_id,
                    inputs={
                        "data": f"{model_name}.in_memory.inf_predictions",
                        "run_id": f"{model_name}.training.run_id",
                    },
                    outputs=f"{model_name}.inference.inf_predictions",
                    name="add_run_id_to_predictions",
                    tags=["inference: predictions"],
                )